# Feature Engineering
The script is mainly operate the big file feature engineering, try to make the process as efficient as possible.


In [2]:
import gc
gc.collect()

0

In [3]:
import pandas as pd
import datetime as dt
import pytz
from sqlalchemy import create_engine
from tqdm import tqdm
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define your connection parameters
username = 'bullionbear'
password = 'Sunshine4Jellybean'
host = 'localhost'
port = '5432'
database = 'lynkoraDB'

# Create the connection string
connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'

# Create the database engine
engine = create_engine(connection_string)

# Define your SQL query
n_days = 150
end_time = int(dt.datetime(2024, 5, 25, tzinfo=pytz.UTC).timestamp() * 1000) - 1
start_time = end_time - n_days * 86400_000
query = f'SELECT open_time, open, high, low, close, close_time FROM btcusdt_kline where open_time between {start_time} and {end_time}'

# Read the data in chunks
chunksize = 86400  # Adjust the chunk size as needed
data_frames = []

with tqdm(total=n_days, desc="Processing Chunks") as pbar:
    for chunk in pd.read_sql_query(query, engine, chunksize=chunksize):
        # Process each chunk here (e.g., filter, transform, etc.)
        grouped = chunk.groupby(chunk.index // 30).agg(
            open_time=('open_time', 'first'),
            open=('open', 'first'),
            high=('high', 'max'),
            low=('low', 'min'),
            close=('close', 'last'),
            close_time=('close_time', 'last'),
        )
        data_frames.append(grouped)
        pbar.update(1)  # Update the progress bar
    

# Combine all chunks into a single DataFrame if needed
df = pd.concat(data_frames, ignore_index=True)

df.head()

Processing Chunks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:49<00:00,  3.04it/s]


,open_time,open,high,low,close,close_time
0,1703635200000,42508.93,42530.85,42508.89,42508.90,1703635229999
1,1703635230000,42508.89,42523.17,42508.89,42514.07,1703635259999
2,1703635260000,42514.07,42515.52,42514.06,42515.51,1703635289999
3,1703635290000,42515.51,42531.10,42515.51,42531.10,1703635319999
4,1703635320000,42531.09,42541.10,42531.09,42541.10,1703635349999


In [3]:
df.describe()

,open_time,open,high,low,close,close_time
count,4.320000e+05,432000.000000,432000.000000,432000.000000,432000.000000,4.320000e+05
mean,1.710115e+12,57259.142309,57273.779491,57244.214225,57259.200442,1.710115e+12
std,3.741234e+09,10800.093158,10804.502947,10795.585472,10800.083824,3.741234e+09
min,1.703635e+12,38558.920000,38564.000000,38555.000000,38558.920000,1.703635e+12
25%,1.706875e+12,44537.040000,44548.000000,44528.475000,44537.115000,1.706875e+12
50%,1.710115e+12,61958.870000,61976.395000,61942.645000,61958.870000,1.710115e+12
75%,1.713355e+12,66715.342500,66733.180000,66696.735000,66715.272500,1.713355e+12
max,1.716595e+12,73775.550000,73777.000000,73701.100000,73775.540000,1.716595e+12


In [4]:
import numpy as np
kline = df.copy()
kline.loc[:, "log_price"] = np.log(kline.loc[:, "close"])
kline.loc[:, "log_return"] = kline.loc[:, "log_price"].diff()

In [5]:
for i in range(1, 15):
    window = 6 * 2 ** i
    kline.loc[:, f'ma_{window}'] = kline.loc[:, 'log_return'].rolling(window=window).mean()
    kline.loc[:, f'ma_{window}_std'] = kline.loc[:, 'log_return'].rolling(window=window).std()
    

In [6]:
n_compound = 5
for i in range(1, n_compound):
    window = 6 * 2 ** i
    kline.loc[:, f'ma_{window}_shift'] = kline.loc[:, f'ma_{window}'].shift(-window)
    kline.loc[:, f'ma_{window}_std_shift'] = kline.loc[:, f'ma_{window}_std'].shift(-window)

In [12]:
kline['reward'] = kline.loc[:, f'ma_{12}_shift'] / kline.loc[:, f'ma_{12}_std_shift']

In [13]:
for i in range(2, n_compound):
    window = 6 * 2 ** i
    kline['reward'] += kline.loc[:, f'ma_{window}_shift'] / kline.loc[:, f'ma_{window}_std_shift']

In [22]:
kline = kline.dropna()

In [26]:
[f"ma_{6 * 2 ** i}" for i in range(1, 15)] + ["open_time", "log_return"] 

['ma_12',
 'ma_24',
 'ma_48',
 'ma_96',
 'ma_192',
 'ma_384',
 'ma_768',
 'ma_1536',
 'ma_3072',
 'ma_6144',
 'ma_12288',
 'ma_24576',
 'ma_49152',
 'ma_98304',
 'open_time',
 'log_return']

In [29]:
interest_cols = ["open_time", "log_return"] + [f"ma_{6 * 2 ** i}" for i in range(1, 15)] + [f"ma_{6 * 2 ** i}_std" for i in range(1, 15)]+ ["reward"]
kline = kline[interest_cols]

In [30]:
kline

,open_time,log_return,ma_12,ma_24,ma_48,ma_96,ma_192,ma_384,ma_768,ma_1536,...,ma_384_std,ma_768_std,ma_1536_std,ma_3072_std,ma_6144_std,ma_12288_std,ma_24576_std,ma_49152_std,ma_98304_std,reward
98304,1706584320000,-5.911780e-04,-0.000176,-0.000073,-0.000080,0.000004,-1.167265e-06,0.000014,0.000013,0.000024,...,0.000416,0.000360,0.000449,0.000381,0.000352,0.000356,0.000434,0.000423,0.000544,-0.086728
98305,1706584350000,-1.596206e-04,-0.000131,-0.000098,-0.000079,0.000005,-1.351793e-07,0.000016,0.000012,0.000024,...,0.000414,0.000360,0.000449,0.000380,0.000352,0.000356,0.000434,0.000423,0.000544,0.042062
98306,1706584380000,-1.672270e-04,-0.000144,-0.000069,-0.000067,0.000005,-9.606951e-07,0.000016,0.000012,0.000024,...,0.000414,0.000360,0.000449,0.000379,0.000352,0.000356,0.000434,0.000423,0.000544,-0.124733
98307,1706584410000,2.297266e-07,-0.000115,-0.000067,-0.000065,0.000005,-1.577935e-06,0.000016,0.000012,0.000023,...,0.000414,0.000360,0.000449,0.000379,0.000352,0.000356,0.000434,0.000423,0.000544,0.162060
98308,1706584440000,2.627727e-04,-0.000077,-0.000049,-0.000054,0.000002,-7.439697e-07,0.000018,0.000013,0.000024,...,0.000414,0.000359,0.000449,0.000379,0.000352,0.000356,0.000434,0.000423,0.000544,0.028482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431899,1716592170000,-3.257258e-04,-0.000048,-0.000034,-0.000004,-0.000005,-1.049663e-05,-0.000013,0.000009,0.000013,...,0.000296,0.000334,0.000398,0.000395,0.000492,0.000503,0.000430,0.000433,0.000466,-0.391298
431900,1716592200000,-3.862367e-04,-0.000081,-0.000051,-0.000005,-0.000007,-1.160367e-05,-0.000014,0.000009,0.000013,...,0.000297,0.000334,0.000398,0.000394,0.000491,0.000503,0.000430,0.000433,0.000466,-0.111888
431901,1716592230000,1.647582e-04,-0.000067,-0.000046,-0.000010,-0.000005,-1.505616e-05,-0.000013,0.000010,0.000013,...,0.000297,0.000334,0.000398,0.000393,0.000491,0.000503,0.000430,0.000433,0.000466,-0.248963
431902,1716592260000,2.328519e-05,-0.000065,-0.000045,-0.000006,-0.000006,-1.664952e-05,-0.000020,0.000009,0.000013,...,0.000263,0.000333,0.000398,0.000393,0.000491,0.000503,0.000430,0.000433,0.000466,-0.340593
